In [1]:
#ランタイムのタイプをGPUに変更
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install -U --quiet langchain-google-genai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 710.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/

In [2]:
import os
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
import pickle
path_w1 = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_texts.pkl'
with open(path_w1, mode='rb') as f:
    texts = pickle.load(f)
path_w2 = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_metadatas.pkl'
with open(path_w2, mode='rb') as f:
    metadatas = pickle.load(f)

In [5]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

<ipython-input-5-7d781a8d0969>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
 

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [6]:
!pip uninstall faiss-cpu

In [7]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00


In [8]:
vectorstore = FAISS.from_texts(texts,embedding, metadatas)
#実行は3分ほど

In [9]:
vectorstore.save_local('/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu./vectorstore')
#vectorestoreを保存

In [11]:
vectorstore = FAISS.load_local('/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu./vectorstore', embedding,allow_dangerous_deserialization=True)
#vectorestoreを読み込み

In [22]:
retriever = vectorstore.as_retriever()
#retrieverの指定　デフォルトは類似度検索
#引数をsearch_kwargs={'filter': {'授業形態':'講義'}}とすると絞り込める
template = """
ユーザー:Questionに当てはまる科目を複数個挙げて、それぞれの科目の概要を説明してください。

{context}

{question}

システム:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)
#プロンプトの作成

os.environ['GOOGLE_API_KEY'] = '********************************'
#https://ai.google.dev/からAPIのキーを取得
llm = ChatGoogleGenerativeAI(model='gemini-pro')
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

q = "源氏物語について学べる科目はなんですか？"
ans = qa.invoke(q)

print(ans['result'])

docs = ans['source_documents']
for d in docs:
    print()
    print(d.page_content)


> Finished chain.
* 日本古典講読論I
* 日本古典講読論II

(科目ナンバリング)U-HUM3224333LJ36U-HUM6624333LJ36(科目名)日本古典講読論II(英訳)ReadinJapaneseClassicsII(所属部局)(職名)(氏名)人間・環境学研究科教授長谷川千尋(配当学年)2-4回生(単位数)2単位(開講年度・開講期)2024・後期(曜時限)火3(授業形態)講義(使用言語)日本語(授業の概要・目的)「源氏見ざる歌詠みは遺恨の事也」とは藤原俊成の言葉であるが、歌人や連歌師は、『源氏物語』のどのような言葉・場面に魅力を感じ、その詩想を養ったのであろうか。このような観点から、文安六年（1446）、祐倫の手になる『山頂湖面抄』という源氏物語梗概書を読む。併せて『源氏物語』本文や、それを享受した作例にも触れ、巻ごとに源氏絵を参照する。(到達目標)・講義中に取り扱った『源氏物語』本文、及びその梗概書や享受作品の内容を理解し、説明できる。・歌人・連歌師による『源氏物語』享受の特色を理解し、説明できる。(授業計画と内容)1．『山頂湖面抄』概説・蛍2．常夏・篝火3．野分・行幸4．藤袴・真木柱5．梅枝・藤裏葉6．若菜上・若菜下7．柏木・横笛8．鈴虫・夕霧9．御法・幻（匂宮・紅梅・竹河は割愛する）10．橋姫・椎本11．総角・早蕨12．宿木・東屋13．浮舟・蜻蛉14．手習・夢浮橋〈学期末試験〉15.フィードバック(履修要件)特になし(成績評価の方法・観点)定期試験（筆記）により、到達目標の達成度を評価する。なお、定期試験が対面で実施できなくなった場合は、オンラインで試験を実施する。到達目標についてＡ＋：すべての観点においてきわめて高い水準で目標を達成している。Ａ：すべての観点において高い水準で目標を達成している。Ｂ：すべての観点において目標を達成している。Ｃ：大半の観点において学修の効果が認められ、目標をある程度達成している。Ｄ：目標をある程度達成しているが、更なる努力が求められる。Ｆ：学修の効果が認められず、目標を達成したとは言い難い。(教科書)使用しないプリント配布。(参考書等)授業中に紹介する(授業外学修（予習・復習）等)授業前に『源氏物語』の当該巻を通読しておくとよい。講義内容の復習。

(科目ナンバリング)U-HUM322433